In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import numpy as np
import pandas as pd
import models.baselinelgb as lgb
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [3]:
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
print('Training data shape: ', app_train.shape)
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
print('Testing data shape: ', app_test.shape)

Training data shape:  (307511, 122)
Testing data shape:  (48744, 121)


In [4]:
training_data = app_train.dropna(axis='columns')
training_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_INCOME_TYPE,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,Working,...,0,0,0,0,0,0,0,0,0,0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,State servant,...,0,0,0,0,0,0,0,0,0,0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,Working,...,0,0,0,0,0,0,0,0,0,0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,Working,...,0,0,0,0,0,0,0,0,0,0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,Working,...,0,0,0,0,0,0,0,0,0,0


In [5]:
testing_data = app_test.dropna(axis='columns')
testing_data.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_GOODS_PRICE,NAME_INCOME_TYPE,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,450000.0,Working,...,0,0,0,0,0,0,0,0,0,0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,180000.0,Working,...,0,0,0,0,0,0,0,0,0,0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,630000.0,Working,...,0,0,0,0,0,0,0,0,0,0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,1575000.0,Working,...,0,0,0,0,0,0,0,0,0,0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,625500.0,Working,...,0,0,0,0,0,0,0,0,0,0


In [6]:
y = training_data['TARGET']

# Remove the feature_labels and target
X = training_data.drop(columns=['SK_ID_CURR', 'TARGET'])
test_ids = testing_data['SK_ID_CURR']
X_submit = testing_data.drop(columns=['SK_ID_CURR'])

# One hot encode features
X = pd.get_dummies(X)
X_submit = pd.get_dummies(X_submit)

# Align the dataframes by the columns
X, X_submit = X.align(X_submit, join='inner', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Extract feature names
feature_names = list(X.columns)

# Convert to np arrays
X = np.array(X)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
# Final data
X_submit = np.array(X_submit)

In [7]:
X_train

array([[0.        , 0.0372079 , 0.184     , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00812356, 0.11572921, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01714973, 0.03370787, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.04723699, 0.26966292, ..., 0.        , 0.        ,
        0.        ],
       [0.07142857, 0.01714973, 0.05617978, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01464246, 0.17098427, ..., 0.        , 0.        ,
        0.        ]])

In [8]:
[y_train.sum() / y_train.shape[0], y_test.sum() / y_test.shape[0]]

[0.08060225683717602, 0.0808553812534145]

In [9]:
# assert not np.any(np.isnan(X_train))
# X_train[np.isnan(X_train)] = 0
# X_test[np.isnan(X_test)] = 0

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [43]:
model = Sequential()
model.add(Dense(128, input_dim=139, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [44]:
import tensorflow as tf
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC()])

In [45]:
model.fit(X_train, y_train, epochs=20, batch_size=100)

Epoch 1/20
1538/1538 [==============================] - 2s 1ms/step - loss: 0.3674 - auc_6: 0.5294
Epoch 2/20
1538/1538 [==============================] - 2s 1ms/step - loss: 0.2947 - auc_6: 0.5741
Epoch 3/20
1538/1538 [==============================] - 2s 1ms/step - loss: 0.2798 - auc_6: 0.5931
Epoch 4/20
1538/1538 [==============================] - 2s 1ms/step - loss: 0.2748 - auc_6: 0.6087
Epoch 5/20
1538/1538 [==============================] - 2s 1ms/step - loss: 0.2741 - auc_6: 0.6120
Epoch 6/20
1538/1538 [==============================] - 2s 1ms/step - loss: 0.2734 - auc_6: 0.6179
Epoch 7/20
1538/1538 [==============================] - 2s 1ms/step - loss: 0.2725 - auc_6: 0.6253
Epoch 8/20
1538/1538 [==============================] - 2s 1ms/step - loss: 0.2725 - auc_6: 0.6286
Epoch 9/20
1538/1538 [==============================] - 2s 1ms/step - loss: 0.2725 - auc_6: 0.6281
Epoch 10/20
1538/1538 [==============================] - 2s 1ms/step - loss: 0.2725 - auc_6: 0.6271
Epoch 11/

In [46]:
y_predict = model.predict(X_test)

In [47]:
# y_predict = model.predict(X_test)
valid_auc = metrics.roc_auc_score(y_test, y_predict)
print(valid_auc)

0.65702190514457
